In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann
!pip install tf_keras
!pip install pandas


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement scann (from versions: none)
ERROR: No matching distribution found for scann

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [55]:
import pprint
import tempfile

from typing import Dict, Text

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [56]:
data = pd.read_csv("/Users/zack/Documents/GLASS/completed-components.csv")

In [76]:
dataset = tf.data.experimental.make_csv_dataset(
    "/Users/zack/Documents/GLASS/completed-components.csv",
    batch_size=1,  # Adjust batch size as needed
    select_columns=['user_id', 'title'],  # Only load the desired columns
    label_name=None,
    num_epochs=1,
    shuffle=False
)

components = tf.data.experimental.make_csv_dataset(
    "/Users/zack/Documents/GLASS/completed-components.csv",
    batch_size=1,  # Adjust batch size as needed
    select_columns=['title'],  # Only load the desired columns
    label_name=None,
    num_epochs=1,
    shuffle=False
)

def dataset_map_and_squeeze(features):
    # Use tf.squeeze() to remove the (None,) dimension and make it ()
    title = tf.squeeze(features['title'])  # Squeeze to shape ()
    user_id = tf.squeeze(features['user_id'])  # Squeeze to shape ()
    title = tf.ensure_shape(title, ())
    user_id = tf.ensure_shape(user_id, ())
    return {"title": title, "user_id": user_id}

def components_map_and_squeeze(features):
    # Use tf.squeeze() to remove the (None,) dimension and make it ()
    title = tf.squeeze(features['title'])  # Squeeze to shape ()
    title = tf.ensure_shape(title, ())
    return title

# mapped_dataset = dataset.map(lambda x: {
#     "title": x["title"],
#     "user_id": x["user_id"],
# })

mapped_dataset = dataset.map(dataset_map_and_squeeze)

components = components.map(components_map_and_squeeze)


In [77]:
print(type(mapped_dataset))
print(mapped_dataset)
print(type(components))
print(components)

<class 'tensorflow.python.data.ops.map_op._MapDataset'>
<_MapDataset element_spec={'title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None)}>
<class 'tensorflow.python.data.ops.map_op._MapDataset'>
<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [78]:
tf.random.set_seed(42)
shuffled = mapped_dataset.shuffle(600, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(480)
test = shuffled.skip(480).take(120)

In [81]:
components_titles = components.batch(60)
user_ids = mapped_dataset.batch(6000).map(lambda x: x["user_id"])

# print("list(components_titles) = ", list(components_titles))
print("list(user_ids) = ", list(user_ids))

unique_component_titles = np.unique(np.concatenate(list(components_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_component_titles[:10]

list(user_ids) =  [<tf.Tensor: shape=(575,), dtype=string, numpy=
array([b'Ai-Che', b'Ai-Che', b'Ai-Che', b'Ai-Che', b'Ai-Che', b'Ai-Che',
       b'Ai-Che', b'Ai-Che', b'Ai-Che', b'Ai-Che', b'Ai-Che', b'Ai-Che',
       b'Ai-Che', b'Ai-Che', b'Ci-Che', b'Ci-Che', b'Ci-Che', b'Ci-Che',
       b'Ci-Che', b'Ci-Che', b'Ci-Che', b'Ci-Che', b'Ci-Che', b'Ci-Che',
       b'Ci-Che', b'Ci-Che', b'Cu-Com', b'Cu-Com', b'Cu-Com', b'Cu-Com',
       b'Cu-Com', b'Cu-Com', b'Cu-Com', b'Cu-Com', b'Cu-Com', b'Cu-Com',
       b'Cu-Com', b'Cu-Com', b'Cn-Che', b'Cn-Che', b'Cn-Che', b'Cn-Che',
       b'Cn-Che', b'Cn-Che', b'Cn-Che', b'Cn-Che', b'Cn-Che', b'Cn-Che',
       b'Cn-Che', b'Cn-Che', b'Cn-Che', b'Dt-Che', b'Dt-Che', b'Dt-Che',
       b'Dt-Che', b'Dt-Che', b'Dt-Che', b'Dt-Che', b'Dt-Che', b'Dt-Che',
       b'Dt-Che', b'Dt-Che', b'Dt-Che', b'Dt-Che', b'Dt-Che', b'Dg-Com',
       b'Dg-Com', b'Dg-Com', b'Dg-Com', b'Dg-Com', b'Dg-Com', b'Dg-Com',
       b'Dg-Com', b'Dg-Com', b'Dg-Com', b'Dg-Com', b'Dg-Co

array([b' NYU CBE Department Research Seminars',
       b"'girlsnight' Fashion Week Rave for Gaza",
       b'2022 Boston Computer Vision Summit',
       b'2023 - 2024 Undergraduate Student Leader of The Year',
       b'2023 Breast Cancer March',
       b'2023 International Conference on Information Technology and Contemporary Sports (TCS)',
       b'2023 SHPE National Convention',
       b'2024 18th European Conference on Antennas and Propagation (EuCAP)',
       b'3D Printed Biomedical Devices VIP',
       b'8th Annual GI Web Conference: Embankment, Dams, and Slopes Extereme Event Research and Trends'],
      dtype=object)

In [82]:
user_ids_vocabulary=tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None)

In [83]:
component_titles_vocabulary=tf.keras.layers.StringLookup(vocabulary=unique_component_titles, mask_token=None)

In [84]:
import tensorflow_recommenders as tfrs

In [85]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
component_model = tf.keras.Sequential([
    component_titles_vocabulary,
    tf.keras.layers.Embedding(component_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    components.batch(128).map(component_model)
  )
)

In [86]:
class GLASSComponentModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      component_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and component representations.
    self.user_model = user_model
    self.component_model = component_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    component_embeddings = self.component_model(features["title"])

    return self.task(user_embeddings, component_embeddings)

In [111]:
# Create a retrieval model.
model = GLASSComponentModel(user_model, component_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(mapped_dataset.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    components.batch(100).map(lambda title: (title, model.component_model(title))))

# Get some recommendations.

_, titles = index(np.array(["S -Bio"]))
print(f"Top 10 recommendations for user: {titles[0, :10]}")

Epoch 1/3
1/1 [==============================] - 1s 636ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0139 - factorized_top_k/top_5_categorical_accuracy: 0.2070 - factorized_top_k/top_10_categorical_accuracy: 0.5061 - factorized_top_k/top_50_categorical_accuracy: 0.8800 - factorized_top_k/top_100_categorical_accuracy: 0.9496 - loss: 9971.3643 - regularization_loss: 0.0000e+00 - total_loss: 9971.3643
Epoch 2/3
1/1 [==============================] - 0s 57ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0122 - factorized_top_k/top_5_categorical_accuracy: 0.1757 - factorized_top_k/top_10_categorical_accuracy: 0.4243 - factorized_top_k/top_50_categorical_accuracy: 0.6974 - factorized_top_k/top_100_categorical_accuracy: 0.8017 - loss: 12509.9482 - regularization_loss: 0.0000e+00 - total_loss: 12509.9482
Epoch 3/3
1/1 [==============================] - 0s 66ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0209 - factorized_top_k/top_5_categorical_accuracy: 0.2365 - f